In [3]:
import sqlite3
import tkinter as tk
from tkinter import messagebox

conn = sqlite3.connect('students.db')


conn.execute('''CREATE TRIGGER IF NOT EXISTS enrollments_fk1
               BEFORE INSERT ON enrollments
               FOR EACH ROW
               BEGIN
                 SELECT RAISE(ABORT, 'Foreign key violation: course_id does not exist')
                 WHERE NOT EXISTS (SELECT 1 FROM courses WHERE id = NEW.course_id);
               END''')
def join_tables():
    # Retrieve the table names selected by the user
    table1 = table1_dropdown.get()
    table2 = table2_dropdown.get()

    # Execute a SQL query to join the tables
    cursor = conn.cursor()
    

    if table1 == "courses" and table2 == "instructors":
        cursor.execute(f"SELECT {table1}.*, {table2}.email, {table2}.office FROM {table1} LEFT JOIN {table2} ON {table1}.instructor = {table2}.name")
        
    elif table1 == "students" and table2 == "enrollments":
        cursor.execute(f"SELECT {table1}.*, {table2}.course_id AS {table2}_course_id, {table2}.grade AS {table2}_grade FROM {table1} LEFT JOIN {table2} ON {table1}.id = {table2}.student_id")
    
    elif table1 == "courses" and table2 == "enrollments":
        cursor.execute(f"SELECT {table1}.*, {table2}.student_id AS {table2}_student_id, {table2}.grade AS {table2}_grade FROM {table1} LEFT JOIN {table2} ON {table1}.id = {table2}.course_id")
    
    elif table1 == "students" and table2 == "instructors":
        cursor.execute(f"SELECT {table1}.*, {table2}.email, {table2}.office FROM {table1} LEFT JOIN {table2} ON {table1}.id = {table2}.id")
    else:
        messagebox.showerror("Invalid Tables", "Please select valid tables")
        return


    records = cursor.fetchall()

    # Clear the listbox
    listbox.delete(0, tk.END)

    # Add each record to the listbox
    for record in records:
        # Create a new record with the repeated columns removed
        new_record = []
        for i in range(len(record)):
            if i == 1 and table2 in record:
                continue
            elif i == 3 and table2 in record:
                continue
            elif i == 4 and table2 in record:
                continue
            else:
                new_record.append(record[i])
        listbox.insert(tk.END, new_record)
        
window = tk.Tk()
window.title("MySQL Left Join")

# Create the dropdown menus for table selection
tables1 = ["students", "courses"]
tables2 = ["instructors", "enrollments"]
table1_dropdown = tk.StringVar(window)
table1_dropdown.set(tables1[0])
table2_dropdown = tk.StringVar(window)
table2_dropdown.set(tables2[1])
table1_menu = tk.OptionMenu(window, table1_dropdown, *tables1)
table2_menu = tk.OptionMenu(window, table2_dropdown, *tables2)
table1_menu.grid(row=0, column=0, padx=5, pady=5)
table2_menu.grid(row=0, column=1, padx=5, pady=5)

# Create the join button
join_button = tk.Button(window, text="Join Tables", command=join_tables)
join_button.grid(row=0, column=2, padx=5, pady=5)

# Create the listbox to display the join results
listbox = tk.Listbox(window, width=80)
listbox.grid(row=1, column=0, columnspan=3, padx=5, pady=5)

window.mainloop()
